In [26]:
# imports

import pandas as pd
import yfinance as yf
import yahoofinancials as YahooFinancials
from datetime import date, datetime, timedelta

In [27]:
# Define the lookup for the stock (name, price, symbol)

import requests
import urllib.parse


api_key = 'pk_35ab12ce73ea4cf38d7641e8d3b01de7'

def lookup(symbol, api_key="pk_35ab12ce73ea4cf38d7641e8d3b01de7"):
    """Look up quote for symbol."""

    # Contact API
    try:
        response = requests.get(
            f"https://cloud.iexapis.com/stable/stock/{urllib.parse.quote_plus(symbol)}/quote?token={api_key}")
        response.raise_for_status()
    except requests.RequestException:
        return None

    # Parse response
    try:
        quote = response.json()
        return {
                "name": quote["companyName"],
                "price": float(quote["latestPrice"]),
                "symbol": quote["symbol"]
                }
    except (KeyError, TypeError, ValueError):
        return None


In [28]:
# input the stock symbol

symbol = input("Enter a stock symbol: ")

In [29]:
# get todays date

today = date.today()
print(today)

2022-03-28


In [30]:
# print the lookup for the sumbol

print(lookup(symbol))

{'name': 'Apple Inc', 'price': 174.72, 'symbol': 'AAPL'}


In [31]:
# date mathematics
# another import line because the code doesn't work without it

from datetime import datetime, timedelta

timedifference = input("Enter a time difference in months: ")
timedifference = int(timedifference)

# timedifference is the number of months
timediffdays = timedifference * 30
# timediffdays is the number of days

startdate = datetime.today() - timedelta(days=timediffdays)
print(startdate)


2022-02-26 13:50:15.429374


In [32]:
# gets data from yahoo finance from a set date to today

stock_df = yf.download(symbol, 
                      start=startdate, 
                      end=today, 
                      progress=False,
)
stock_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-28,163.059998,165.419998,162.429993,165.119995,165.119995,94869100
2022-03-01,164.699997,166.600006,161.970001,163.199997,163.199997,83474400
2022-03-02,164.389999,167.360001,162.949997,166.559998,166.559998,79724800
2022-03-03,168.470001,168.910004,165.550003,166.229996,166.229996,76678400
2022-03-04,164.490005,165.550003,162.100006,163.169998,163.169998,83737200


In [33]:
stock_values_list = stock_df.values.tolist()
print(stock_values_list)

# list of lists of data in the form [[open, high, low, close, volume, adj close]]

[[163.05999755859375, 165.4199981689453, 162.42999267578125, 165.1199951171875, 165.1199951171875, 94869100.0], [164.6999969482422, 166.60000610351562, 161.97000122070312, 163.1999969482422, 163.1999969482422, 83474400.0], [164.38999938964844, 167.36000061035156, 162.9499969482422, 166.55999755859375, 166.55999755859375, 79724800.0], [168.47000122070312, 168.91000366210938, 165.5500030517578, 166.22999572753906, 166.22999572753906, 76678400.0], [164.49000549316406, 165.5500030517578, 162.10000610351562, 163.1699981689453, 163.1699981689453, 83737200.0], [163.36000061035156, 165.02000427246094, 159.0399932861328, 159.3000030517578, 159.3000030517578, 96418800.0], [158.82000732421875, 162.8800048828125, 155.8000030517578, 157.44000244140625, 157.44000244140625, 131148300.0], [161.47999572753906, 163.41000366210938, 159.41000366210938, 162.9499969482422, 162.9499969482422, 91454900.0], [160.1999969482422, 160.38999938964844, 155.97999572753906, 158.52000427246094, 158.52000427246094, 1053